In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import math
import multiprocessing as mp

In [74]:
days_left = 365*1
# days_left = 10

In [2]:
def get_securities(date):
    page_size = 100
    all_rows = []
    start = 0
    url_temp = "https://iss.moex.com/iss/history/engines/stock/markets/bonds/securities.json?date={date}&start={start}&iss.json=extended"
    r = requests.get(url_temp.format(start=start, date=date))
    res = r.json()
    all_rows += res[1]['history'][1]
    total = res[1]['history.cursor'][1][0]['TOTAL']
    pages = math.ceil(total/page_size)+1
    print("Date:{} Total:{}, Pages:{}".format(date, total, pages))
    if total <=100:
        return all_rows
    for page in range(1, pages):
        start = page*page_size
        r = requests.get(url_temp.format(start=start, date=date))
        res = r.json()
        all_rows += res[1]['history'][1]
    return all_rows

# res = get_securities("2020-09-01")



In [80]:
secs = []
td = datetime.today()- timedelta(1)
dates = []
for i in range(days_left):
    dt = td - timedelta(i)
    dates.append(dt.strftime('%Y-%m-%d'))
    

In [ ]:
# pool = mp.Pool(mp.cpu_count())
# results = pool.map(get_securities, [date for date in dates])
# pool.close()

Date:2021-09-03 Total:2376, Pages:25
Date:2021-07-19 Total:2344, Pages:25
Date:2021-09-02 Total:2378, Pages:25
Date:2021-07-18 Total:0, Pages:1
Date:2021-07-17 Total:0, Pages:1
Date:2021-07-16 Total:2341, Pages:25
Date:2021-09-01 Total:2379, Pages:25
Date:2021-08-31 Total:2377, Pages:25
Date:2021-07-15 Total:2343, Pages:25
Date:2021-07-14 Total:2349, Pages:25
Date:2021-08-30 Total:2377, Pages:25
Date:2021-08-29 Total:0, Pages:1
Date:2021-08-28 Total:0, Pages:1
Date:2021-08-27 Total:2375, Pages:25
Date:2021-08-26 Total:2373, Pages:25
Date:2021-08-25 Total:2378, Pages:25
Date:2021-07-13 Total:2341, Pages:25
Date:2021-08-24 Total:2377, Pages:25
Date:2021-07-12 Total:2336, Pages:25
Date:2021-08-23 Total:2375, Pages:25
Date:2021-07-11 Total:0, Pages:1
Date:2021-07-10 Total:0, Pages:1
Date:2021-07-09 Total:2336, Pages:25
Date:2021-08-22 Total:0, Pages:1
Date:2021-08-21 Total:0, Pages:1
Date:2021-08-20 Total:2372, Pages:25
Date:2021-07-08 Total:2337, Pages:25
Date:2021-08-19 Total:2376, Pages

Date:2021-02-09 Total:2270, Pages:24
Date:2021-02-08 Total:2263, Pages:24
Date:2020-12-24 Total:2243, Pages:24
Date:2021-02-07 Total:0, Pages:1
Date:2021-02-06 Total:0, Pages:1
Date:2021-02-05 Total:2255, Pages:24
Date:2020-12-23 Total:2241, Pages:24
Date:2020-12-22 Total:2233, Pages:24
Date:2020-12-21 Total:2226, Pages:24
Date:2020-12-20 Total:0, Pages:1
Date:2020-12-19 Total:0, Pages:1
Date:2020-12-18 Total:2225, Pages:24
Date:2021-02-04 Total:2256, Pages:24
Date:2020-12-17 Total:2224, Pages:24
Date:2020-12-16 Total:2227, Pages:24
Date:2021-02-03 Total:2261, Pages:24
Date:2020-12-15 Total:2224, Pages:24
Date:2021-02-02 Total:2258, Pages:24
Date:2021-02-01 Total:2256, Pages:24
Date:2021-01-31 Total:0, Pages:1
Date:2021-01-30 Total:0, Pages:1
Date:2021-01-29 Total:2254, Pages:24
Date:2020-12-14 Total:2223, Pages:24
Date:2020-12-13 Total:0, Pages:1
Date:2020-12-12 Total:0, Pages:1
Date:2020-12-11 Total:2213, Pages:24
Date:2020-12-10 Total:2205, Pages:24
Date:2020-12-09 Total:2206, Pages

In [ ]:
secs = []
for r in results:
    secs += r

In [3]:
df = pd.DataFrame(get_securities("2021-09-02"))
df.to_excel("moex_bonds.xlsx", index=False)

Date:2021-09-02 Total:2378, Pages:25
